In [0]:
import sqlite3
import pandas as pd
import re


def create_conn():
    conn = sqlite3.connect(":memory:")
    return conn
    
def get_cursor(conn):
    return conn.cursor()
    
def create_table_students(schema, conn):
    creation_string = ''
    for k,v in schema.items():
        creation_string = creation_string + k + " " + v + ", "
    creation_string_f = creation_string[:len(creation_string)-2]
    try:
        cur = get_cursor(conn)
        cur.execute(f"""
        create table students 
        ({creation_string_f})
        """)
        print(f'Created table students with the schema: {creation_string_f}')
        conn.commit()
    except Exception as e:
        print(e)
        
def insert_records_into_students(records, conn):
        cols = list(records.keys())
        num_records = len(records[cols[0]])
        print(f"Going to Insert {num_records} record(s)")
        for i in range(num_records):
            record_string = ''
            for k in records.keys():
                record_string = record_string + str(records[k][i]) + "," 
                record_string_f = record_string[:len(record_string)-1]
            try:
                cur = get_cursor(conn)
                cur.execute(f"""
                insert into students values
                ({",".join(["?"] * len(cols))})
                """, tuple(record_string_f.split(',')))
                conn.commit()
                print(f"Record {str(i+1)} inserted: {record_string_f}")
            except Exception as e :
                print(f"Failed to insert record {str(i+1)}: {record_string_f}")
                print(e)
        print("-"*80)
                
def query_students_table(conn, query= None):
    if((query is None) or (len(re.sub(r"\s+", "", query)) == 0)):
        print(f"Displaying all records currently in the students table below")
        print(pd.read_sql_query("select * from students", con= conn))
    else:
        print(f"Below are the results of the custom query : {query}")
        try:
            print(pd.read_sql_query(query, con= conn))
        except Exception as e:
            print(f"Query failed. see the details below")
            print(e)
schema = {
     'roll_no' : 'integer',
     'name' : 'text',
     'subject': 'text', 
     'marks' : 'integer',
     'class': 'integer',
}

records = {
     'roll_no': [1, 2, 3, 4, 5, 2, 3, 4],
     'name': ['Rishabh', 'Abhishek', 'Rahul', 'Rohan', 'Rishabh', 'Abhishek', 'Rahul', 'Rohan'],
     'subject': ['ComputerScience', 'ComputerScience', 'ComputerScience', 'ComputerScience', 'Maths', 'Maths', 'Maths', 'Maths'], 
     'marks' : [95, 95, 90, 90, 94, 90, 89, 84], 
     'class': [12, 11, 11, 12, 12, 11, 11, 12]
}

def main():
    conn = create_conn()



    q = """ 

    """
    create_table_students(schema, conn)
    insert_records_into_students(records, conn)
    query_students_table(conn)
    query_students_table(conn, q)

    conn.close()


if(__name__ == '__main__'):
    main()


Questions

q1: topper for each class (having highest total marks combined)(with and without window functions)

q2: Find name of students with wrongly assigned roll_no(same class, same name, different roll_no)

q3: Validate num of records using a python fuction

q4: Implement update and delete methods

q5: create student class and make if so that the record for that student gets auto inserted into the table

Solutions

In [0]:
select name, class, max(total) as highest 
from
    ( select name, class, sum(marks) as total
    from 
    students
    group by 1, 2) sq1
group by 2 

    custom_q_with_join_and_cond = """
    select distinct name
    from
        (select s1.*, s2.*
        from 
        students s1 join students s2
        on s1.roll_no != s2.roll_no and s1.name = s2.name and s1.class = s2.class
        ) as sq
    """

    def delete_from_students_table(conn, stmnt= None):
    cur = get_cursor(conn)
    if(stmnt is None):
        print("No delete query provided. Deleting all records")
        cur.execute("""
        delete from students where 1=1
        """)
    else:
        cur.execute(stmnt)
    conn.commit()

    def update_students_table(conn, stmnt= None):
        cur = get_cursor(conn)
        if(stmnt is None):
            print("No query provided. Add query in update_q variable and pass it")
            return
        else:
            cur.execute(stmnt)
        conn.commit()

    custom_q_with_self_join_and_filter_after = """
    select distinct name
    from
        (select *
        from (
              select s1.*, s2.roll_no as s2_roll_no, s2.name as s2_name, s2.class as s2_class
              from 
              students s1 cross join students s2
              ) sq1
        where
        roll_no != s2_roll_no and name = s2_name and class = s2_class) sq2
    """

    delete_q = """
    delete from students
    where Name='Abhishek'
    """
    update_q = """
    update students
    set marks=100
    where Name='Rishabh'
    """
    
    create_table_students(schema, conn)
    insert_records_into_students(records, conn)
    query_students_table(conn)
    ts1 = time.time()
    query_students_table(conn, custom_q_with_join_and_cond)
    te1 = time.time()
    print(f"Time took to complete the above query was {(te1-ts1)*1000} ms ")
    ts2 = time.time()
    query_students_table(conn, custom_q_with_self_join_and_filter_after)
    te2 = time.time()
    print(f"Time took to complete the above query was {(te2-ts2)*1000} ms ")
    delete_from_students_table(conn, delete_q)
    query_students_table(conn)
    update_students_table(conn)
    update_students_table(conn, update_q)
    query_students_table(conn)
    delete_from_students_table(conn)
    query_students_table(conn)
    conn.close()
  class student:
    def __init__(self,conn, roll_no, name, subject, marks, clas):
        self.roll_no = roll_no
        self.name = name
        self.subject =subject
        self.marks = marks
        self.clas = clas
        record = {
         'roll_no': [self.roll_no],
         'name': [self.name],
         'subject': [self.subject], 
         'marks' : [self.marks], 
         'class': [self.clas]
        }
        insert_records_into_students(record, conn)
    def __repr__(self):
        return f"student(roll_no={self.roll_no}, name= {self.name}, subject={self.subject}, marks= {self.marks}, class={self.clas})"

In [0]:
import sqlite3
import pandas as pd
import re
import random
import pprint as pp

source_positions_data = {
    'acc_id' : [],
    'isin_code': [],
    'quantity' : [],
    'settlement_dt' : [],
    'currency' : []
}

target_positions_data = {
    'acc_id' : [],
    'isin_code': [],
    'quantity' : [],
    'settlement_dt' : [],
    'currency' : []
}

def generate_data(source_positions_data, no_of_records= 200):
    weights = [1, 2, 45, 1, 1, 1, 1, 1, 2, 45]
    qty = list(range(100, 1000, 200))
    qty.append(None)
    for dt in pd.date_range('2024-01-01', '2025-03-31', no_of_records):
        source_positions_data['settlement_dt'].append(dt.date())
    for _ in range(1, no_of_records+1):
        source_positions_data['acc_id']\
                .append("".join([str(d) for d in random.choices(list(range(0,10)), k=16, weights= weights)]))
        source_positions_data['currency'].append(random.choice(['USD', None, 'EUR', 'INR', 'AUD', 'GBP']))
        if(source_positions_data['currency'][-1] is not None):
            country_prefix = source_positions_data['currency'][-1][0:2]
            source_positions_data['isin_code']\
                .append(country_prefix + \
                        "".join([str(d) for d in random.choices(list(range(0,10)), k=10, weights= weights)]))
        else:
            source_positions_data['isin_code']\
                .append(None)
        source_positions_data['quantity'].append(random.choice(qty))

def create_conn():
    conn = sqlite3.connect(":memory:")
    return conn
    
def get_cursor(conn):
    return conn.cursor()

def create_target_positions_table(target_schema_str, conn):
    try:
        cur = get_cursor(conn)
        cur.execute(f"""
        create table target_positions 
        ({target_schema_str})
        """)
        print(f'Created table target_positions with the schema: {target_schema_str}')
        conn.commit()
    except Exception as e:
        print(e)
    
def write_to_target_positions_table(target_df, conn):
    try:
        return target_df.to_sql(name= 'target_positions', con= conn, if_exists= 'append', index= False)
    except Exception as e:
        print(e)
 
def query_target_positions_table(conn, query= None):
    if((query is None) or (len(re.sub(r"\s+", "", query)) == 0)):
        print(f"Displaying all records currently in the target_positions table below")
        print(pd.read_sql_query("select * from target_positions", con= conn))
    else:
        print(f"Below are the results of the custom query : {query}")
        try:
            print(pd.read_sql_query(query, con= conn))
        except Exception as e:
            print(f"Query failed due to {e} error")    
            
if __name__ == "__main__":
    conn = create_conn()
    
    target_schema_str = "acc_id text, isin_code text, quantity float, settlement_dt date, currency text"
    
    #fake data generation
    generate_data(source_positions_data, no_of_records= 1000)
    generate_data(target_positions_data, no_of_records= 10000)
    # pp.pprint(source_positions_data)
    
    source_df = pd.DataFrame(source_positions_data)
    target_df = pd.DataFrame(target_positions_data)
    
    create_target_positions_table(target_schema_str, conn)
    target_rows_written = write_to_target_positions_table(target_df, conn)
    print("target_rows_written: ", target_rows_written)
    
    #use the below variable for custom query against the target table
    q = """ 

    """
    
    query_target_positions_table(conn, q)
    
    conn.close()

    